In [16]:
import tensorflow as tf
import numpy as np 
import gym 
import networkx as nx
#from tensorflow import keras


In [17]:
#reward function che poi inseriremo nel modello di fit di Keras, 


In [18]:
#proviamo prima una fully connected
n = 5
m = 8
k=n*m
RailCost=0
StopCost=0
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()
#def RewardWrap():

#tf.function
def Outputter(tensor1):
    #count=0
    
    #with tf.compat.v1.Session() as sess:
        #tensor=tf.shape(tensor1).eval(session=sess)
           
    for i in tensor1:
        x=np.random.rand(0,1)
        if(x<i):
            i=1
            #print("ciao")
            #count+=1
        else: 
            i==50
       # print("ciao1")
    return tensor1

class MySoftmax(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MySoftmax, self).__init__() #????
        self.num_outputs = num_outputs
    
    def build(self, input_shape):
         b_init = tf.zeros_initializer()
         self.kernel = self.add_weight("kernel",shape=[int(input_shape[-1]),self.num_outputs])
         self.b = tf.Variable(initial_value=b_init(shape=int(input_shape[-1]), dtype="float32"))
                                                          
    def call(self, input):
        output=(softmax((tf.matmul(input, self.kernel)+self.b) ))
       # print(tf.keras.backend.softmax(tf.matmul(input, self.kernel)+self.b))
        output=Outputter(output)
        output=tf.convert_to_tensor(output)
        return output

        

In [19]:
 
#la mappa è un lattice i cui nodi rappresentano le POSSIBILI stazioni 
MapMatrix= np.zeros(shape=( n, m))
#La così detta amtrice verdhttp://localhost:8888/notebooks/Metropolitan.py.ipynb#e, dato che vogliamo usare keras, possiamo generarla a priori un numero arbitrario di volte e trattare il problema come semi-supervisionato
#in questo caso le sezioni di TravelBank sono le TravelMatrix
NumberOfCities=1000
TravelBank=np.zeros(shape=(NumberOfCities,n*m,n*m,))
for s in range(NumberOfCities):
    TravelBank[s] = np.random.rand(n*m, n*m)
    for i in range(k):
        somma=TravelBank[s,i].sum()
        for j in range(k):
            TravelBank[s,i,j]=TravelBank[s,i,j]/somma
                               
    
TravelBank1=np.zeros(shape=(NumberOfCities,k*k))
for s in range(NumberOfCities):
    for i in range(k):
         TravelBank1[s]=np.matrix.flatten(TravelBank[s])
    
np.shape(TravelBank1)
    


(1000, 1600)

In [20]:
   print("ciao")
batch=40
#Adjecency matrix relativa al grafo corrispondente ai viaggi che vogliono essere percorsi
#(i-j entrata  = probabilità che la persona sulla posizione i voglia andare nella posizione j,
#non necessariamente simmetrica, gli elementi diagonali sono sempre nulli e la matrice è  generalemnte DENSA)
#navigare lo spazio di queste matrici significa navigare lo spazio delle possibili città in cui possiamo definire 
#una griglia fissata di possibili stazioni, dunque la sua esplorazione sarà l'  " ambiente" in cui si muove il nostro agente

StateMatrix = np.zeros(shape=(n*m, n*m))
for i in range(k):
    for j in range(k):
           if(i != j):
                 StateMatrix[i,j]=50

StateGraph=nx.DiGraph(StateMatrix)


ciao


In [27]:

def RewardFunction(TravelMatrix,StateMatrix,RailCost,StopCost):
   # with tf.compat.v1.Session() as sess:
        #init = tf.compat.v1.global_variables_initializer()
        #sess.run(init)
    const=np.zeros(shape=(batch,k,k))
    for i in range(k):
        for j in range(k):
            if(i!=j):
                i=1/50
    const1=tf.constant(const,dtype='float32')
    
    print("aeeee",TravelMatrix)
    TravelMatrix1=tf.zeros(shape=(batch,k*k))
    StateGraph1=np.array([])
    StateMatrix1=np.empty(shape=(batch,k*k))
    StateMatrix2=np.empty(shape=(batch,k,k))
    TravelMatrix2=tf.zeros(shape=(batch,k,k))
    
         
    StateMatrix2= StateMatrix1.reshape((batch,k,k))
    for s in range(batch):
        #TravelMatrix1[s]=(TravelMatrix.numpy())[s]
        StateMatrix=tf.reshape(StateMatrix,shape=(batch,k,k))
        StateMatrix=tf.math.multiply(const1,StateMatrix)
        #StateMatrixCompleted
        TravelMatrix=tf.reshape(TravelMatrix,shape=(batch,k,k))
        print("ciao",StateMatrix[s])
        Reward=0#trasforma le matrici in un vettore e basta 
        #railcost conviene farlo calcolare quando generiamo la matrice di stato in uscita dal NN, dato che dobbiamo comunque assegnare i pesi.
        #StateGraph1=np.append(StateGraph1,nx.DiGraph(StateMatrix[s]))
        for i in range(k):
            
            for j in range((k)):
                #if(i%10==0):
                print(i,j)
                Reward+=(TravelMatrix[s,i,j])/(Dijkstra(StateMatrix,i,j)) - RailCost - StopCost
    return Reward 
#model= tf.keras.models.Sequential()
#si aspetta il prodotto tensoriale delle due matrici 
#in entrata proviamo all'inizio un modello fully connected 
#model.add(tf.keras.layers.Dense(16,activation='relu',input_shape=(n*m,n*m)))
#model.add(tf.keras.layers.Dense((n*m)*(n*m),activation='softmax'))

#+ tutti i layer che vogliamo 
input_tensor=tf.keras.Input(shape=(k*k), batch_size=batch)
x=tf.keras.layers.Dense(16,activation='relu')(input_tensor)
x=tf.keras.layers.Dense(32,activation='relu')(x)
#print(tf.keras.layers.Dense((n*m)*(n*m),activation='softmax')(x))
#x=tf.keras.layers.Dense((n*m)*(n*m),activation='softmax')(x)
#MySoftmax=MySoftmax(16)
#output_tensor=MySoftmax(x)
#l'output ha shape (none,qualcosa,qualcosa), devp riuscire ad accedere ai numeri singoli in qualche modo e fare outputter
x=(tf.keras.layers.Dense((k*k),activation='softmax'))(x)
#print(tf.convert_to_tensor(x))
#output_tensor=(tf.function(Outputter(x)))
print(x) 
output_tensor=x
model=tf.keras.Model(input_tensor,output_tensor)
 
def CustomReward(y,y_predicted,i):
    return -RewardFunction(i,y_predicted,RailCost,StopCost)
def CustomRewardWrap(input_tensor1):
    return CustomReward(0,output_tensor,input_tensor1)
#l'output ha la dimensione della matrice di stato: indica con un numero normalizzato da 0 a 1 la probabilità di modificare l'edge i,j, se l'edge è occuato lo toglierà, altrimenti lo metterà
model.compile(optimizer="Adam",loss=CustomRewardWrap(input_tensor),run_eagerly=True)


#model.compile(optimizer="Adam",loss=lambda y, Output_tensor: -RewardFunction(y,Output_tensor, input_tensor,0,0))

y_train=np.zeros(shape=(NumberOfCities,k*k))
#potrebbe non anfare ancora perchè il modello non è stato buildato, quindi la shape è indefinita.
history = model.fit(TravelBank1, y_train, batch_size=batch, epochs=1)

model.predict(TravelBank1[0])

Tensor("dense_14/Identity:0", shape=(40, 1600), dtype=float32)
aeeee Tensor("input_5:0", shape=(40, 1600), dtype=float32)
ciao Tensor("strided_slice_12:0", shape=(40, 40), dtype=float32)
0 0


ValueError: in converted code:

    <ipython-input-26-46269852164f>:40 Dijkstra  *
        Distances[u]=tempDist

    ValueError: setting an array element with a sequence.


In [ ]:
#oppure potremmo semplicemnte avere l'intera matrice direttamente come output: in tal caso gli stati diventano proprio i valori dell'ambiente(i possibili valori della matrice di Travel)
#praticamente, la matrice in uscita potrebbe comunque essere una softmax, per cui però la probabilità rappresenta le chance che la rispettiva entrata della matrice sia trasformata in un uno, così che il programma tenda a far convergere questo valore a 0.99 per le decisioni migliori.

#a questo punto calcolo la matrice di output e i reward totale associato alla matrice e poi inizializzo di nuovo l'a

In [26]:
#funzione algoritmo dijkstra:
#la facci prima in python, poi vedo se riesco co  K.backend
@tf.function
def Dijkstra(StateMatrix,source,target):
#l'algo calcola un array di distanze(una per ogni nodo del grafo
    Distances=np.array([])
    Queue=np.array([])
    Visited=np.array([])
    for s in range(k):
        if(s != source):
            Distances=np.append(Distances,100000000) 
        else:
            Distances=np.append(Distances,0) 
        Queue=np.append(Queue,s)
   # print(Distances)
   
   
    v=0
    switch=1
    while(len(Queue)!=0 and switch):
        #print(Queue)
        v=int (int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))#devo escludere dalle distanze quell ivisitati?
        if(v not in Visited):
           
            Queue=np.delete(Queue,int (np.where(Queue==int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))[0]))
            Visited=np.append(Visited,v)
      
        #print(Distances)
        #print((np.where(Distances == np.amin(Distances))))
            #else:
                #v=int (np.where(Distances == np.amin(Distances))[0])
                #switch=0
       
        for u in range(k):
            if(StateMatrix[v,u]!=0):
                tempDist=Distances[v]+StateMatrix[v,u]
                   # print(tempDist)
                   # print(v,u)
                if(tempDist<Distances[u]):
                    Distances[u]=tempDist
    #print(Distances)
    return Distances[target]
A=np.zeros(shape=(k,k))
A=np.random.rand(k,k)

A1=nx.DiGraph(A)
print(((nx.algorithms.shortest_paths.weighted.dijkstra_path_length(A1,0,39))))
print(Dijkstra(A,0,39))

0.04011481593337174


ValueError: in converted code:

    <ipython-input-26-46269852164f>:40 Dijkstra  *
        Distances[u]=tempDist

    ValueError: setting an array element with a sequence.


In [25]:
@tf.function
def DijkstraKeras(StateMatrix,source,target):
#l'algo calcola un array di distanze(una per ogni nodo del grafo
    Distances=np.array([])
    Distances1=tf.tf.Variable(Distances,dtype='float32')
    Queue=np.array([])
    Queue1=tf.constant(Queue,dtype='float32')
    Visited=np.array([])
    Visited1=tf.constant(Visited,dtype='float32')
    for s in range(k):
        if(s != source):
            Distances1=tf.concat(Distances,100000000) 
        else:
            Distances1=tf.concat(Distances,0) 
        Queue1=tf.concat(Queue,s)
   # print(Distances)
   
   
    v=0
    switch=1
    while(len(Queue)!=0 and switch):
        #print(Queue)
        v=int (int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))#devo escludere dalle distanze quell ivisitati?
        if(v not in Visited):
           
            Queue=np.delete(Queue,int (np.where(Queue==int (np.where(Distances == np.sort(Distances)[len(Visited)])[0]))[0]))
            Visited=np.append(Visited,v)
      
        #print(Distances)
        #print((np.where(Distances == np.amin(Distances))))
            #else:
                #v=int (np.where(Distances == np.amin(Distances))[0])
                #switch=0
       
        for u in range(k):
            if(StateMatrix[v,u]!=0):
                tempDist=Distances[v]+StateMatrix[v,u]
                   # print(tempDist)
                   # print(v,u)
                if(tempDist<Distances[u]):
                    Distances[u]=tempDist
    #print(Distances)
    return Distances[target]
A=np.zeros(shape=(k,k))
A=np.random.rand(k,k)

A1=nx.DiGraph(A)
print(((nx.algorithms.shortest_paths.weighted.dijkstra_path_length(A1,0,39))))
print(Dijkstra(A,0,39))

0.1290888807408045
0.1290888807408045
